In [1]:
from telethon import TelegramClient, events
from telethon.sessions import StringSession
import serial

# Use your own values from my.telegram.org
api_id = 1123190
api_hash = '3b7e5587b7251109e6d8a4ce89152b49'
string = ""

client = TelegramClient(StringSession(string), api_id, api_hash)
await client.connect()
# await client.send_code_request("886928052495")
await client.start()


# 886928052495

Please enter your phone (or bot token): 886928052495
Please enter the code you received: 68121
Signed in successfully as Muller Lee


In [2]:
from pypinyin import lazy_pinyin

def _modify_string(text):
    def remove_non_anscii(text):
        return ''.join([i if ord(i) < 128 else ' ' for i in text])
    result = remove_non_anscii(''.join(lazy_pinyin(text)))
    result = result.replace('\n','').replace('\r','')
    return result

async def get_dialog_list():
    dialogs = await client.get_dialogs()
    return [[i.id, _modify_string(i.title), i.unread_count] for i in dialogs]

async def get_dialog_list2(offset):
    dialogs = await client.get_dialogs()
    return [str(i.id) for i in dialogs][offset * 5:][:5], [_modify_string(i.title) for i in dialogs][offset * 5:][:5]


async def get_message(dialog_id, offset):
    msg = await client.get_messages(dialog_id, add_offset=offset)
    try:
        return _modify_string(msg[0].text), str(msg[0].sender.is_self)
    except:
        return "NULL", "true"

async def send_message(dialog_id, text):
    await client.send_message(int(dialog_id), text)

def send_stm32(ser, text):
    ser.write(f"{text}\0".encode())


In [ ]:
import serial
import json
ser = serial.Serial('COM5', 115200, timeout=1,)
stop = False
try:
    while not stop:
        while ser.in_waiting:
            text = ser.readline().decode()
            try:
                data = json.loads(text)
                print(data)
                if data['cmd'] == 'sendmsg':
                    await send_message(data['id'], data['msg'])

                elif data['cmd'] == 'getlist':
                    ids, data = await get_dialog_list2(int(data['offset']))
                    response = {
                        "type": "list",
                        "data": data,
                        "data2": ids
                    }
                    response = json.dumps(response)
                    print(response)
                    send_stm32(ser, response)

                elif data['cmd'] == 'getmsg':
                    data, isme = await get_message(int(data['id']), int(data['offset']))
                    response = {
                        "type": "msg",
                        "data": data,
                        "data2": isme
                    }
                    response = json.dumps(response)
                    print(response)
                    send_stm32(ser, response)

                elif data['cmd'] == 'stop':
                    stop = True
                    break

            except:
                pass
    ser.close()
except:
    ser.close()


{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["Telegram", "Muller Lee", "ceng  ", "zhengyoukai", "gugula"], "data2": ["777000", "478970169", "636940658", "665155360", "-256089738"]}
{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["Telegram", "Muller Lee", "ceng  ", "zhengyoukai", "gugula"], "data2": ["777000", "478970169", "636940658", "665155360", "-256089738"]}
{'cmd': 'getmsg', 'id': '478970169', 'offset': '0'}
{"type": "msg", "data": "aybca  a", "data2": "True"}
{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["Telegram", "Muller Lee", "ceng  ", "zhengyoukai", "gugula"], "data2": ["777000", "478970169", "636940658", "665155360", "-256089738"]}
{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["Telegram", "Muller Lee", "ceng  ", "zhengyoukai", "gugula"], "data2": ["777000", "478970169", "636940658", "665155360", "-256089738"]}
{'cmd': 'getmsg', 'id': '478970169', 'offset': '0'}
{"type": "msg", "data": "aybca  a", "data2": "True"}
{'cmd'

{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["Muller Lee", "ceng  ", "Telegram", "zhengyoukai", "gugula"], "data2": ["478970169", "636940658", "777000", "665155360", "-256089738"]}
{'cmd': 'getmsg', 'id': '478970169', 'offset': '0'}
{"type": "msg", "data": "fuck you", "data2": "True"}
{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["Muller Lee", "ceng  ", "Telegram", "zhengyoukai", "gugula"], "data2": ["478970169", "636940658", "777000", "665155360", "-256089738"]}
{'cmd': 'getmsg', 'id': '636940658', 'offset': '0'}
{"type": "msg", "data": "apple", "data2": "True"}
{'cmd': 'sendmsg', 'id': '636940658', 'msg': 'fuck you'}
{'cmd': 'getmsg', 'id': '636940658', 'offset': '0'}
{"type": "msg", "data": "fuck you", "data2": "True"}
{'cmd': 'getlist', 'offset': '0'}
{"type": "list", "data": ["ceng  ", "Muller Lee", "Telegram", "zhengyoukai", "gugula"], "data2": ["636940658", "478970169", "777000", "665155360", "-256089738"]}
{'cmd': 'getmsg', 'id': '636940658', 'offse

In [ ]:
ser.close()